Ant Colony - Laboratório IA
Pedro Santos e Vinicius Nascimento

In [63]:
from City import City
from Ridge import Ridge
import numpy as np
import random
import math
import csv

# Variáveis Globais
alpha = 1
beta = 1
cities_array = []
ants = []
ridges = np.empty((6, 6), dtype=Ridge) # Vai da cidade 0 até a cidade 32

In [73]:
# Construindo o vetor de cidades, e instanciando os objetos City para salvar as coordenadas de cada cidade

def build_cities():
    global cities_array
    with open('C:\\Users\\pedro\\OneDrive\\Documentos\\CEFETMG\\2023.1\\Inteligência Artificial\\Laboratório\\AI-Algorithms\\Ant Colony - Teste\\Colonia.csv', "r") as csvfile:
        filereader = csv.reader(csvfile, delimiter=';')
        next(filereader, None)
        for row in filereader:
            id, x, y = row
            id = int(id)
            x = float(x)
            y = float(y)
            cities_array.append(City(id, x, y))

In [65]:
# Construindo o vetor de formigas, cada posição do vetor equivale a uma formiga, e guarda as cidades vistadas por ela

def build_ants_initial_position():
    global ants, cities_array
    for i in cities_array:
        ants.append([i.get_id()])

In [66]:
# Cálculo da distância entre duas cidades usando função euclidiana

def distance(x1:float,x2:float,y1:float,y2:float) -> float:
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [67]:
# Criando uma matriz de arestas para salvar todas as distâncias entre todas as cidades

def build_ridges():
    global ridges, cities_array

    for i in range(len(ridges)):
        for j in range(len(ridges)):
            if i != j:
                ridges[i][j] = Ridge(i, j, 1, distance(cities_array[i].get_x(), cities_array[j].get_x(), cities_array[i].get_y(), cities_array[j].get_y()))
            else:
                ridges[i][j] = Ridge(i, j, 1, 0)

In [95]:
# Calcular a probabilidade de cada formiga ir para cada cidade

def propability(possible_cities:list, ant_cities:list) -> list :

    sum = 0
    cities_propability = []

    print("Caminho Percorrido: ")
    print(ant_cities)

    print("Possíveis próximas cidades: ")
    print(possible_cities)

    # Cidade atual da formiga
    present_city = ant_cities[-1]

    # Calculando o somatório das possíveis rotas
    for i in range(len(possible_cities)):

        ridge = ridges[present_city][possible_cities[i]]
        ph = ((ridge.get_pheromone())**alpha) * ((ridge.get_heuristic())**beta)
        sum = sum + ph

    # Calculando a propabilidade de cada rota
    for i in range(len(possible_cities)):

        prob = 0

        ridge = ridges[present_city][possible_cities[i]]
        p = ridge.get_pheromone()
        h = ridge.get_heuristic()

        prob = ((p)**alpha) * ((h)**beta) / sum

        cities_propability.append(prob)

    return cities_propability



In [108]:
# Calcular a próxima cidade a ser visitada por cada formiga

def select_next_city():
    global ants, cities_array, ridges

    # Probability deve receber a lista de possíveis cidades a serem visitadas pela formiga e a lista das cidade já visitadas

    # Percorrendo o vetor de formigas
    for i in range(1, len(ants), 1):

        possible_cities = []
        next_city = 0        

        # Percorrendo o vetor de cidades para cada fromiga
        for j in range(1, len(cities_array), 1):

             # Checando se a cidade j já foi visitada pela formiga i
            if cities_array[j].get_id() not in ants[i]:
                possible_cities.append(cities_array[j].get_id())
        
        # Calculando a probabilidade para cada cidade não visitada pela formiga i
        probabilities = propability(possible_cities, ants[i])

        # Linkar a cidade com a sua probabilidade de ser visitada, tupla (cidade, probabilidade)
        city_probability = []
        for i in range(len(possible_cities)):
            city_probability.append((possible_cities[i], probabilities[i]))

        print("Probabilidae de cada rota: ")
        print(city_probability)

        # Aplicando o método da roleta para selecionar a próxima cidade -> Revisar Lógica
        sum_city_probability = []
        for i in range(len(city_probability)):
            if i != 0:
                sum_city_probability.append(city_probability[i][1] + sum_city_probability[i-1])
            else:
                sum_city_probability.append(city_probability[i][1])

        print(sum_city_probability)

        r = random.uniform(0, 1)

        for i in range(len(sum_city_probability)):
            if r < sum_city_probability[i]:
                next_city = possible_cities[i]
                break

        print("Próxima Cidade Escolhida: ")
        print(next_city, " ", r)
        print("\n")


In [109]:
def main():

    global ridges, cities_array, ants

    # Limpando as variáveis
    cities_array.clear(), ants.clear()

    # DEBUG
    # print(cities_array)
    # print(ants)
    # print(ridges)

    # Construindo o vetor de cidades
    build_cities()

    # DEBUG
    # for i in range(len(cities_array)):
    #     print(cities_array[i].get_id(), end=" ")
    #     print(cities_array[i])

    # Construindo o vetor de formigas
    build_ants_initial_position()

    # DEBUG
    # print(*ants)

    # Construindo a matriz de arestas
    build_ridges()

    # DEBUG
    # for i in range(len(ridges)):
    #     for j in range(len(ridges)):
    #         print(ridges[i][j])
    #         print(ridges[i][j].get_city_origin(), ridges[i][j].get_city_destiny(), end="    ")
    #     print("\n")

    # Calculando próxima cidade a ser visitada
    select_next_city()

    # DEBUG
    

In [110]:
main()

# 0,2  0,6  0,8  0,9

Caminho Percorrido: 
[1]
Possíveis próximas cidades: 
[2, 3, 4, 5]
Probabilidae de cada rota: 
[(2, 0.21787473044651565), (3, 0.4357494608930313), (4, 0.22458013169599), (5, 0.12179567696446303)]
[0.21787473044651565, 0.653624191339547, 0.878204323035537, 1.0]
Próxima Cidade Escolhida: 
4   0.6928554084835541


Caminho Percorrido: 
[2]
Possíveis próximas cidades: 
[1, 3, 4, 5]
Probabilidae de cada rota: 
[(1, 0.18062751578809855), (3, 0.18062751578809855), (4, 0.364066397179629), (5, 0.27467857124417394)]
[0.18062751578809855, 0.3612550315761971, 0.7253214287558261, 1.0]
Próxima Cidade Escolhida: 
5   0.8787880657383842


Caminho Percorrido: 
[3]
Possíveis próximas cidades: 
[1, 2, 4, 5]
Probabilidae de cada rota: 
[(1, 0.2346628043052356), (2, 0.1173314021526178), (4, 0.3532142940207653), (5, 0.2947914995213812)]
[0.2346628043052356, 0.3519942064578534, 0.7052085004786187, 0.9999999999999999]
Próxima Cidade Escolhida: 
5   0.7723292722659816


Caminho Percorrido: 
[4]
Possíveis próxim